In [1]:
###Imports

from numpy import *
from numpy.random import *
import numpy as np
import numpy.linalg as npl
import Bio.PDB as PDB

# Kronecker product, needed to calculate Matrix Normal pdf
from theano.tensor.slinalg import kron as kron_product

In [2]:
### Defining helper functions

def sph_rand(n=3):
    """
    Return a random vector on the n-dimensional sphere S.
    """
    # Simply generate n gaussian numbers and normalize
    # From Knuth's book
    a=zeros(n, 'd')
    for i in range(0,n):
        a[i]=normal(0,1)
    return a/sqrt(sum(a*a))


def random_rotation():
    """
    Generate a random 3D rotation matrix.
    
    Requires the function sph_rand
    
    Adapted from L. Devroye, Non-Uniform Random Variate Generation (page 607,
    available online at U{http://www-cgrl.cs.mcgill.ca/~luc/rnbookindex.html}),
    by setting s=1 to avoid generating roto-inversions.
    """

    # Random rotation around z
    x,y=sph_rand(2)
    rz=array(((x,-y,0), (y, x, 0), (0,0,1)), 'd')
    
    # Random rotation around y
    x,y=sph_rand(2)
    ry=array(((x,0,y), (0,1,0), (-y, 0, x)), 'd')
    
    # Random rotation around x
    x,y=sph_rand(2)
    rx=array(((1,0,0), (0,x,y), (0, -y, x)), 'd')
    
    # Combine the three and return
    rnd_rot = rz @ ry @ rx
    return rnd_rot

def is_orthogonal(RM):
    """
    Returns boolean T/F if matrix is orthogonal or not (M.t = M^-1 in orthogonal matricies)
    """
    #transpose
    a = RM.T

    #inverse
    b = npl.inv(RM)
    
    #Checks the differnce element wise is < 0.01
    #boolean, does transpose ~= inverse (a=b element wise?)
    c = np.allclose(a,b, atol=0.01)
    print("and orthogonality is...")
    print(c)
    return

In [3]:
### Pre-calcualtions
N=10 #Constant represents the number of vectors.

# Generating Average structure (Mu)
# generates a N(5) x 3 roation matrix
# *100 is scaling so noise has only minor effect (as intended)
Mu=100*randn(N,3) 
print("MEAN STRUCTURE")
print(Mu)

## making two different roation matricies
rot1 = random_rotation()
rot1_d = npl.det(rot1)

print("ROTATION 1:")
print(rot1)
print("The determinant is...")
print (rot1_d)
is_orthogonal(rot1)


rot2 = random_rotation()
rot2_d = npl.det(rot2)

print("ROTATION 2:")
print(rot2)
print("The determinant is...")
print(rot2_d)
is_orthogonal(rot2)


## Rotated structures + noise
# This is our basic equation, where each structure is a perturbation of Mu.
T_struct1 = dot(Mu,rot1)+randn(N,3)
T_struct2 = dot(Mu,rot2)+randn(N,3)

print("TRUE STRUCTURE 1")
print(T_struct1)

print("TRUE STRUCTURE 2")
print(T_struct2)

# Flatten matricies, for easier calcualtions later
Mu_f=array(Mu.flat)
T_struct1_f=array(T_struct1.flat)
T_struct2_f=array(T_struct2.flat)

MEAN STRUCTURE
[[ -85.93588533 -107.94714045 -163.36321294]
 [-134.95975393   40.86906714 -109.31105022]
 [ 106.83138782  -59.82855606 -138.44486559]
 [ 181.1115063    80.93163056   40.54126708]
 [ -29.436116     57.61810783  180.95212025]
 [ -10.88027243 -199.86691062  -79.54029616]
 [ -56.24244218  -66.12376952   48.52596099]
 [  25.90530637   22.36420204    3.68021426]
 [ 130.02171796  -32.82413417   52.12409277]
 [ 100.84118252   16.77161834  -13.13504648]]
ROTATION 1:
[[-0.93146599  0.34408968  0.11820914]
 [-0.2128628  -0.25190827 -0.94405066]
 [-0.29506023 -0.90451341  0.30788788]]
The determinant is...
1.0
and orthogonality is...
True
ROTATION 2:
[[ 0.1545133  -0.97572814  0.1551781 ]
 [-0.94389429 -0.19218473 -0.26856767]
 [ 0.29187189 -0.10497445 -0.95067932]]
The determinant is...
1.0
and orthogonality is...
True
TRUE STRUCTURE 1
[[ 151.06897669  144.85394992   41.82731342]
 [ 148.3739493    42.20339353  -90.44390583]
 [ -46.10372013  174.42384726   27.91446964]
 [-197.37629

In [4]:
### Imports for theano - THESE MUST BE SEPARATE
import numpy as np
import numpy.random as npr
import numpy.linalg as npl
import pymc3
from pymc3 import *
from pymc3.math import *
import theano.tensor as TT
import theano as T
from theano import function
from theano import pp
# Kronecker product, needed to calculate Matrix Normal pdf
from theano.tensor.slinalg import kron as kron_product

### Here we define the Theano model we will call later.
with Model() as model:
    
### Defining functions for the  model
         
    def make_rotation(i):
        """
        Determines quaternions from a unifrom prior and forms from them; a rotation matrix.
        Prior over the rotation is included within
        """
        ### Rotation prior
        #uniform prior used so as to not draw preference to one particular parameter
        #this is the continuous , uniform, log-liklihood
        
        # the first argument states that i will be the name of the rotation made
        xr = Uniform('x'+str(i), 0, 1, testval=0.5, shape=3) # Note shape is 3-Dimensional
        
        #Background for this is in the paper provided
        theta1 = 2*np.pi*xr[1]
        theta2 = 2*np.pi*xr[2]
        
        r1 = TT.sqrt(1-xr[0])
        r2 = TT.sqrt(xr[0])

        # Difficult to check the quaternion length is 1 (unit quaternion) - difficult
        qx = TT.cos(theta2)*r2
        qy = TT.sin(theta1)*r1
        qz = TT.cos(theta1)*r1
        qw = TT.sin(theta2)*r2        
    
        ###The rotation matrix from the quaternions
        # initializes a matrix (here 3x3) with 1's on the diagonal
        R = T.shared(np.eye(3))
    
        #filling the rotation matrix
        # https://www.flipcode.com/documents/matrfaq.html#Q54
        # Row one
        R = TT.set_subtensor(R[0,0], 1-(2*TT.sqr(qy))-(2*TT.sqr(qz))) 
        R = TT.set_subtensor(R[0,1], (2*(qx*qy))-(2*(qz*qw))) 
        R = TT.set_subtensor(R[0,2], (2*(qx*qz))+(2*(qy*qw))) 
        # Row two
        R = TT.set_subtensor(R[1,0], (2*(qx*qy))+(2*(qz*qw))) 
        R = TT.set_subtensor(R[1,1], 1-(2*TT.sqr(qx))-(2*TT.sqr(qz))) 
        R = TT.set_subtensor(R[1,2], (2*(qy*qz))-(2*(qx*qw))) 
        # Row three
        R = TT.set_subtensor(R[2,0], (2*(qx*qz))-(2*(qy*qw))) 
        R = TT.set_subtensor(R[2,1], (2*(qy*qz))+(2*(qx*qw))) 
        R = TT.set_subtensor(R[2,2], 1-(2*TT.sqr(qx))-(2*TT.sqr(qy))) 
        return R
    
    #### NOISE NEGATION
    ## Generating the standard covariance matrix
    # U of MatrixNormal -> generate standard covariance matrix
    packed_L = LKJCholeskyCov('packed_L', n=N, eta=2., sd_dist=HalfCauchy.dist(0.5))
    
    # Convert a packed triangular matrix into a two dimensional array.
    L = expand_packed_triangular(N, packed_L)
    U = L.dot(L.T)
    
    # V of MatrixNormal -> generate standard diagonal cov mat
    sv=HalfNormal("sv", sd=1)
    V=sv*TT.eye(3)
    
    VU=kron_product(V,U) 
    
    ### ROTATION    
    r1 = Deterministic('r1',make_rotation("ONE"))
    r2 = Deterministic('r2',make_rotation("TWO"))
    
    ### FORMING THE STRUCTURES
    #predicted mean observed structure for the two structures to be superpositioned??
    obs_struct1=TT.dot(Mu, r1) 
    obs_struct2=TT.dot(Mu, r2)
    #here we have the two structures (no noise)
    
    ### Flattening for computation
    obs_struct1f=TT.flatten(obs_struct1)
    obs_struct2f=TT.flatten(obs_struct2)    
    
    ### MAP of rotation priors 
    ## this is the liklihood
    x_obs = MvNormal("x_obs", mu=obs_struct1f, cov=VU, observed=T_struct1_f)
    y_obs = MvNormal("y_obs", mu=obs_struct2f, cov=VU, observed=T_struct2_f)
    
    ### Setting some of the final values to be in the trace and thus printable
    g1 = Deterministic('g1', x_obs)
    g2 = Deterministic('g2', y_obs)

In [5]:
### Calling Theano the model & processing outputs
map_estimate = find_MAP(maxeval=10000, model=model) # ML estimarion (liklihood x prior ) mind max

#print("THIS IS THE MAP ESTIMATE")
# print(map_estimate)

logp = 28.823, ||grad|| = 1,902.5: 10001it [00:18, 549.79it/s]                           



In [6]:
### Assigning variables from the MAP estimate
x1 = map_estimate['xONE']
x2 = map_estimate['xTWO']

r1 = map_estimate['r1']
r2 = map_estimate['r2'] 
r1_d = npl.det(r1)
r2_d = npl.det(r2) 

g1 = map_estimate['g1']
g2 = map_estimate['g2']

In [7]:
### Comparisons

# print("Comparing true and estimated structures")
# print("TRUE STRUCTURE 1")
# print(T_struct1)
# print("Guess at Structure 1")
# print(g1)

# print("TRUE STRUCTURE 2")
# print(T_struct2)
# print("Guess at Structure 2")
# print(g2)

print("Comparing true and estimated rotations")

print("True R1")
print(rot1)
print("MAP r1")
print(r1)
print("The determinant is...")
print(r1_d)
is_orthogonal(r1)

print("True R2")
print(rot2)   
print("MAP r2")
print(r2)
print("The determinant is...")
print(r2_d)
is_orthogonal(r2)

Comparing true and estimated rotations
True R1
[[-0.93146599  0.34408968  0.11820914]
 [-0.2128628  -0.25190827 -0.94405066]
 [-0.29506023 -0.90451341  0.30788788]]
MAP r1
[[-0.93088866  0.34120832  0.13047297]
 [-0.22456534 -0.25280071 -0.94109628]
 [-0.28812622 -0.90535556  0.31195287]]
The determinant is...
1.0
and orthogonality is...
True
True R2
[[ 0.1545133  -0.97572814  0.1551781 ]
 [-0.94389429 -0.19218473 -0.26856767]
 [ 0.29187189 -0.10497445 -0.95067932]]
MAP r2
[[ 0.15221597 -0.97566456  0.15782576]
 [-0.94458909 -0.19059905 -0.26725167]
 [ 0.29082942 -0.10840052 -0.95061432]]
The determinant is...
1.0
and orthogonality is...
True
